In [1]:
pip install gradio

  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/16/9b/46c017d6bbc60ccb13ef51af4b07dbb288b9080ea3170b8d47beee7842b6/gradio-4.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for altair<6.0,>=4.2.0 from https://files.pythonhosted.org/packages/c5/e4/7fcceef127badbb0d644d730d992410e4f3799b295c9964a172f92a469c7/altair-5.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for fastapi from https://files.pythonhosted.org/packages/ad/b9/b7ea33663daffa9db94119ea2a3df8f97bdca297024145fe79a5a13d37af/fastapi-0.105.0-py3-none-any.whl.metadata
  Using cached ffmpy-0.3.1.tar.gz (5.5 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for gradio-client==0.7.3 from https://files.pythonhosted.org/packages/78/52/a96eada27a2f711464c4a8c85a6110d46e35034cd2108640980c1fa4e8bb/gradio_client-0.7.3-py3-none-any.whl.metadata
  Obtaining dependency inform

DEPRECATION: Loading egg at c:\users\public\kp\anaconda3\lib\site-packages\pycocotools-2.0-py3.11-win-amd64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.3 requires pydantic<2.0, but you have pydantic 2.5.2 which is incompatible.
s3fs 2023.4.0 requires fsspec==2023.4.0, but you have fsspec 2023.12.2 which is incompatible.


In [1]:
import gradio as gr

In [2]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
from keras.applications.xception import Xception #to get pre-trained model Xception
from keras.applications.xception import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.text import Tokenizer #for text tokenization
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import add
from keras.models import Model, load_model
from keras.layers import Input, Dense #Keras to build our CNN and LSTM
from keras.layers import LSTM, Embedding, Dropout
from tqdm import tqdm_notebook as tqdm #to check loop progress
tqdm().pandas()
import PIL.Image

C:\Users\KRIDAY PARMAR\AppData\Local\Temp\ipykernel_28004\1123008542.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm().pandas()


0it [00:00, ?it/s]

In [3]:
from keras.preprocessing.image import load_img
def extract_features(filename):
    model=Xception(include_top=False, pooling="avg")
    image = load_img(filename , target_size=(299,299))
    image = np.array(image)
    
    if image.shape[2] == 4:
        image = image[..., :3]
    image = np.expand_dims(image, axis=0)
    image = image / 127.5
    image = image - 1.0
    feature = model.predict(image)
    return feature

def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo, sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

In [8]:
def get_image_caption(filename,model):
    model = load_model(r"C:\Users\KRIDAY PARMAR\Desktop\Google Describe Function-Kriday Parmar\models\model30k2_2.h5")
    tokenizer = load(open(r"C:\Users\KRIDAY PARMAR\Desktop\Google Describe Function-Kriday Parmar\tokenizer30k2.p",'rb'))
    photo = extract_features(filename,)
    desc = generate_desc(model , tokenizer , photo , 72)
    desc = desc.replace('start','')
    desc = desc.replace('end','')
    return desc.title()

In [9]:
demo = gr.Interface(get_image_caption, gr.Image(type="filepath"),"text")
demo.launch()

C:\Users\Public\KP\anaconda3\Lib\site-packages\gradio\utils.py:840: UserWarning: Expected 2 arguments for function <function get_image_caption at 0x00000210FA3391C0>, received 1.
  warnings.warn(
C:\Users\Public\KP\anaconda3\Lib\site-packages\gradio\utils.py:844: UserWarning: Expected at least 2 arguments for function <function get_image_caption at 0x00000210FA3391C0>, received 1.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


C:\Users\Public\KP\anaconda3\Lib\site-packages\gradio\helpers.py:835: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")



1/1 [==============================] - 2s 2s/step
